In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# End to End Bert Classifier(for task A), and Feature extraction

This notebook contains the first submission to the SemEval Leaderboard. It is fine-tuned Bert with a Classifier. The F1 score on the leaderboard is 0.596. 

The notebook also contains the code for the feature embeddings that will be concatenated with the resnet features. 

In [ ]:
!pip install transformers


     |████████████████████████████████| 3.5 MB 15.8 MB/s 
     |████████████████████████████████| 895 kB 83.1 MB/s 
     |████████████████████████████████| 6.8 MB 97.8 MB/s 
     |████████████████████████████████| 596 kB 73.6 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import os
import sys
import shutil

from transformers import AdamW

In [ ]:


df_raw= pd.read_csv("/content/drive/MyDrive/TRAINING/training.csv", sep='\t', error_bad_lines=False)
df_test_raw = pd.read_csv("/content/drive/MyDrive/TRIAL DATASET/trial.csv", sep='\t', error_bad_lines=False)
df_val_raw = pd.read_csv("/content/drive/MyDrive/temp/test.csv", sep='\t', error_bad_lines=False)

#df_test_raw = pd.read_csv("/content/drive/MyDrive/semeval-trial.csv", sep='\t', error_bad_lines=False)


In [ ]:
df_test_raw.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,Text Transcription
0,28.jpg,0,0,0,0,0,"not now, dad. We should burn Jon Snow. stop it..."
1,30.jpg,0,0,0,0,0,there may have been a mixcommunication with th...
2,33.jpg,0,0,0,0,0,i shouldn't have sold my boat
3,58.jpg,1,0,0,0,1,"Bitches be like, It was my fault i made him mad"
4,89.jpg,0,0,0,0,0,find a picture of 4 girls together on FB make ...


In [ ]:
data = [df_raw["misogynous"], df_raw["Text Transcription"]]
headers = ["category", "text"]
df = pd.concat(data, axis=1, keys=headers)

In [ ]:
df.head()

,category,text
0,0,Milk Milk.zip
1,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,0,BREAKING NEWS: Russia releases photo of DONALD...
3,0,MAN SEEKING WOMAN Ignad 18 O
4,0,Me explaining the deep lore of. J.R.R. Tolkein...


In [ ]:
df_val_raw

,file_name,Text Transcription
0,15236.jpg,FACEBOOK SINGLES GROUPS BELIKE WHEN A NEW WOMA...
1,15805.jpg,"SO, IF YOU'RE A FEMINIST HOW CAN YOU EAT DAIRY?"
2,16254.jpg,WHEN A CUTE GIRL LEFT YOUR MESSAGE ON SEEN
3,16191.jpg,Photographing something you want to show every...
4,15952.jpg,HEY BABE CAN YOU MAKE ME A SANDWICH? Hey babe ...
...,...,...
995,15591.jpg,IT'S NOT YOUR FAULT You didn't design the dres...
996,15049.jpg,THINK ABOUT HOW MUCH BETTER HER SKIN IS BREATH...
997,15363.jpg,THE STEREOTYPES ARE TRUE F SHE DOES HAVE A TIG...
998,15199.jpg,DRAWS NAKED PICTURES OF BLACK WOMEN 00 0000 GE...


In [ ]:
data_test = [df_test_raw["misogynous"], df_test_raw["Text Transcription"]]
headers_test = ["category", "text"]
df_test = pd.concat(data_test, axis=1, keys=headers_test)

In [ ]:
data_val = [ df_val_raw["Text Transcription"]]
headers_val = [ "text"]
df_val = pd.concat(data_val, axis=1, keys=headers_val)

In [ ]:
df_val.head()

,text
0,FACEBOOK SINGLES GROUPS BELIKE WHEN A NEW WOMA...
1,"SO, IF YOU'RE A FEMINIST HOW CAN YOU EAT DAIRY?"
2,WHEN A CUTE GIRL LEFT YOUR MESSAGE ON SEEN
3,Photographing something you want to show every...
4,HEY BABE CAN YOU MAKE ME A SANDWICH? Hey babe ...


In [ ]:
df.replace([0,1],["normal", "misogynous"], inplace=True)
df_test.replace([0,1],["normal", "misogynous"], inplace=True)

In [ ]:
df.head()

,category,text
0,normal,Milk Milk.zip
1,misogynous,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,normal,BREAKING NEWS: Russia releases photo of DONALD...
3,normal,MAN SEEKING WOMAN Ignad 18 O
4,normal,Me explaining the deep lore of. J.R.R. Tolkein...


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = 'BREAKING NEWS: Russia releases photo of DONALD'
bert_input = tokenizer(example_text,padding='max_length', max_length = 100, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

tensor([[  101, 26660, 12420,  2428, 15740, 26546, 18019,   131,  2733,  6596,
          6307,  1104,   141, 11414, 12507,  2137,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
#output_dir = '/content/drive/MyDrive/semeval/checkpoints/'

# Create output directory if needed
#if not os.path.exists(output_dir):
    #os.makedirs(output_dir)

#print("Saving model to %s" % output_dir)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'normal':0,
          'misogynous':1,

          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class TestDataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)

        return batch_texts

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased',output_hidden_states=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        pooled_output = output[1]
        hidden_states = output[2]
        #hidden_states = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)[2]
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
# output_model = '/content/drive/MyDrive/semeval/bert.pth'

# # save
# def save(model, optimizer):
#     # save
#     torch.save({
#         'model_state_dict': model.bert.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict()
#     }, output_model)

# save(model, optimizer)


In [ ]:
print(model.bert)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
print(model)

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=64)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    #output_model = '/content/drive/MyDrive/semeval/bert-full.pth'
    output_model = r'/content/drive/MyDrive/semeval/transformers/bert'
    

    def save(model, optimizer, epoch):
    # save
        save_dir = os.path.join( output_model, "model_{}.hd5".format(epoch))
        torch.save({
            'model_state_dict': model.bert.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }, save_dir)



    

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            save(model, optimizer, epoch_num)


    

In [ ]:
# def evaluate(model, test_data):

#     test = Dataset(test_data)

#     test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

#     use_cuda = torch.cuda.is_available()
#     device = torch.device("cuda" if use_cuda else "cpu")

#     if use_cuda:

#         model = model.cuda()

#     total_acc_test = 0
#     with torch.no_grad():

#         for test_input, test_label in test_dataloader:

#               test_label = test_label.to(device)
#               mask = test_input['attention_mask'].to(device)
#               input_id = test_input['input_ids'].squeeze(1).to(device)

#               output = model(input_id, mask)

#               acc = (output.argmax(dim=1) == test_label).sum().item()
#               total_acc_test += acc
    
#     print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
def evaluate_val(model, test_data):

    test = TestDataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    #layers = {}
    predictions=[] #heres where ur results will be stored
    feature_vec = []
    #pool_hidden_features = []
    #hidden_layers = []
    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input in test_dataloader:

              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              outputs = model(input_id, mask)

              x = model.bert(input_id, mask)
                
              logits = outputs[0]
              #logits2 = x[0]

                #Move logits and labels to CPU
              logits = logits.detach().cpu().numpy()
              layer1 = x[-1][0].detach().cpu().numpy().tolist()
              #layer2 = x[-2][0].detach().cpu().numpy().tolist()
              ##layer3 = x[-3][0].detach().cpu().numpy().tolist()
              #layer4 = x[-4][0].detach().cpu().numpy().tolist()

              #layers['l1'] = layer1
              #layers['l2'] = layer2
              #layers['l3'] = layer3
              #layers['l4'] = layer4


              #pool_feature = outputs[1]
              #pool_features = pool_features.detach().cpu().numpy()

              #embeddings = outputs[2][1:]
              #embeddings = embeddings.detach().cpu().numpy()

              hidden_states = x[2]
              pool_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
              #pool_output = pool_output[:, 0, :]
              #pool_output = self.dropout(pool_output)
              # classifier of course has to be 4 * hidden_dim, because we concat 4 layers
              #logits = self.classifier(pooled_output)




              #last_4_layers = [np.concatenate(x[-4:])]
              #last_4_layers = last_4_layers.detach().cpu().numpy()
              #n_features.append(last_4_layers)
              #print(logits2)


                # Store predictions and true labels
              predictions.append(logits)
              feature_vec.append(layer1)
              #pool_hidden_features.append(pool_features)
              #pool_hidden_features.append(embeddings)



    #return predictions , feature_vec, layers
    return  predictions, pool_output


In [ ]:
#                   lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
#                 )

In [ ]:
model = BertClassifier()

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# output_model = '/content/drive/MyDrive/semeval/bert.pth'

# # save
# def save(model, optimizer):
#     # save
#     torch.save({
#         'model_state_dict': model.bert.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict()
#     }, output_model)

# save(model, optimizer)


In [ ]:
# checkpoint = torch.load(output_model, map_location='cpu')
# y = model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
LR = 5e-5

In [ ]:
EPOCHS = 2
model = BertClassifier()
LR = 5e-5
              
train(model, df, df_test, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 157/157 [01:51<00:00,  1.40it/s]


Epochs: 1 | Train Loss:  0.010 | Train Accuracy:  0.609 | Val Loss:  0.011 | Val Accuracy:  0.800


100%|██████████| 157/157 [01:51<00:00,  1.40it/s]


Epochs: 2 | Train Loss:  0.008 | Train Accuracy:  0.753 | Val Loss:  0.011 | Val Accuracy:  0.820


In [ ]:
PATH = '/content/drive/MyDrive/semeval/transformers/bert/model_7.hd5'


In [ ]:
checkpoint = torch.load(PATH, map_location='cpu')
optimizer = Adam(model.parameters(), lr= LR)
model.bert.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
model


BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
df_val

,text
0,FACEBOOK SINGLES GROUPS BELIKE WHEN A NEW WOMA...
1,"SO, IF YOU'RE A FEMINIST HOW CAN YOU EAT DAIRY?"
2,WHEN A CUTE GIRL LEFT YOUR MESSAGE ON SEEN
3,Photographing something you want to show every...
4,HEY BABE CAN YOU MAKE ME A SANDWICH? Hey babe ...
...,...
995,IT'S NOT YOUR FAULT You didn't design the dres...
996,THINK ABOUT HOW MUCH BETTER HER SKIN IS BREATH...
997,THE STEREOTYPES ARE TRUE F SHE DOES HAVE A TIG...
998,DRAWS NAKED PICTURES OF BLACK WOMEN 00 0000 GE...


In [ ]:
weight_val, pool_layers = evaluate_val(model, df_val)
#a = evaluate_val(model, df_val)

In [ ]:
len(weight_val[1])

1

In [ ]:
pool_layers.shape

torch.Size([1, 128, 3072])

In [ ]:
pool_layers = pool_layers.cpu().numpy()

In [ ]:
type(pool_layers)

numpy.ndarray

In [ ]:
pool_layers.shape

(1, 3072)

In [ ]:
print(pool_layers)

[[ 0.4421799  -0.1421821  -1.4271173  ...  0.06536753 -0.25038546
   0.7731819 ]]


In [ ]:
type(pool_layers)

torch.Tensor

In [ ]:
len(weight_val)

1000

In [ ]:
df_weights_val = pd.DataFrame(weight_val[1])

In [ ]:
df_weights_val.to_csv('/content/drive/MyDrive/semeval/transformers/bert-weights-final.csv', index=False)

In [ ]:
f = evaluate_val(model, df)
#f[1] contains the feature embeddings
#f[0] contains the predictions

IndexError: ignored

In [ ]:
#f[1] contains the feature embeddings
print(f[1])

NameError: ignored

In [ ]:
print(len(f[1]))

NameError: ignored

In [ ]:
#preds is the list of predictions on the validation set. 
preds = []
for i in f[0]:
    preds.append(np.argmax(i))
    

In [ ]:
print(len(preds))

1000


In [ ]:
y_pred = pd.DataFrame(preds)

In [ ]:
y_pred

,0
0,1
1,1
2,1
3,1
4,0
...,...
995,1
996,1
997,0
998,1


In [ ]:
df_val_raw.head()
files = df_val_raw["file_name"]

In [ ]:
final = pd.concat([files,y_pred], axis=1)

In [ ]:
#Constructing a dataframe in accordance to the submission format 
final


,file_name,0
0,15236.jpg,1
1,15805.jpg,1
2,16254.jpg,1
3,16191.jpg,1
4,15952.jpg,0
...,...,...
995,15591.jpg,1
996,15049.jpg,1
997,15363.jpg,0
998,15199.jpg,1


In [ ]:
final.columns = ['file', 'result']

In [ ]:
final['result'].unique()

array([1, 0])

In [ ]:
final['result'].dtype

dtype('int64')

In [ ]:
final

,file,result
0,15236.jpg,1
1,15805.jpg,1
2,16254.jpg,1
3,16191.jpg,1
4,15952.jpg,0
...,...,...
995,15591.jpg,1
996,15049.jpg,1
997,15363.jpg,0
998,15199.jpg,1


In [ ]:
final.to_csv('/content/drive/MyDrive/semeval/submissions/submission.txt', sep = '\t', index=False, header=None)

In [ ]:
df_weights = pd.DataFrame(f[1])
df_weights
#constructing a dataframe of the feature embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.507860,0.052226,0.252734,0.219285,-0.425712,0.887047,-0.602590,-0.983628,-0.241495,0.682547,0.074750,0.452623,-0.989412,-0.399007,0.773873,0.592190,0.509831,-0.318230,-0.683249,0.460346,-0.670997,-0.711586,-0.053397,0.979357,-0.883201,-0.109165,0.651916,0.918744,-0.075874,0.059464,0.069130,-0.785004,0.815737,-0.894670,0.189896,0.174002,0.968200,0.018603,0.499969,-0.941572,...,-0.975840,-0.798884,0.620456,0.733614,0.171998,-0.984356,0.220376,0.336950,-0.821727,-0.955048,-0.721735,0.468211,0.550267,-0.786267,0.116378,0.387517,0.407393,-0.295528,-0.585464,0.815502,0.462020,0.133411,0.003314,-0.783406,-0.761178,-0.149426,0.499699,0.671024,0.264369,-0.999076,-0.194231,-0.691786,0.582272,-0.640601,-0.481741,-0.778416,-0.260559,0.919651,-0.913012,0.303546
1,0.514229,-0.037721,-0.286987,0.347404,-0.370704,0.829875,-0.808047,-0.588387,0.154858,0.504220,0.151436,0.365838,-0.863317,-0.903889,0.404005,0.709321,-0.154199,0.227257,-0.633431,0.951637,-0.716272,-0.817945,-0.351245,0.730850,-0.681420,-0.350962,0.689397,0.919156,-0.219996,0.168511,-0.118238,-0.390384,-0.002613,-0.854289,0.308718,-0.078911,0.857830,0.014735,-0.567219,-0.788534,...,-0.528083,-0.659409,0.565871,-0.274952,0.136521,-0.276735,0.033173,0.272068,-0.361711,0.079562,-0.430931,0.350926,0.064518,-0.843960,-0.263167,0.135315,0.829328,-0.369796,-0.806278,-0.814041,0.609094,-0.087777,0.004907,-0.683240,-0.661963,0.201647,0.196113,0.688244,0.448831,-0.782595,-0.707918,0.046895,-0.006138,0.147284,-0.393747,-0.542385,-0.162506,0.931410,-0.539365,0.094459
2,0.723133,0.029558,-0.131565,0.400288,-0.572151,0.859443,-0.823067,-0.988142,-0.133315,0.557257,-0.708329,-0.032219,-0.997287,-0.790908,0.867830,0.712760,0.048770,0.096581,-0.321740,0.647618,-0.789871,-0.674510,-0.105316,0.968911,-0.906571,-0.364679,0.218643,0.858617,0.390644,0.352524,0.103851,-0.453645,0.621060,-0.781582,0.214703,0.188562,0.965406,0.355869,0.330278,-0.910895,...,-0.943531,-0.808730,0.862751,0.861675,0.392089,-0.988501,0.212667,0.246197,-0.922676,-0.944012,-0.817062,0.716617,0.839261,-0.610813,-0.509790,-0.234934,0.092406,-0.709787,-0.428693,0.792590,0.386477,-0.310569,-0.212075,-0.604163,-0.371400,-0.243780,0.513009,0.606806,0.514728,-0.999389,-0.128001,-0.570646,0.642557,-0.510091,-0.328613,-0.868079,0.081116,0.938043,-0.892591,0.202173
3,0.753092,-0.210068,-0.350908,0.253299,-0.470364,0.981492,-0.948511,-0.997369,0.019649,0.943077,-0.826939,-0.167221,-0.999204,-0.434285,0.955161,0.904565,-0.092440,0.475598,0.142562,0.310258,-0.826051,-0.481503,-0.207713,0.946127,-0.937663,-0.386898,-0.199602,0.736198,-0.296554,0.068298,-0.149380,0.189302,0.943831,-0.724912,-0.009881,0.468497,0.981159,0.315826,0.446506,-0.972032,...,-0.949014,-0.921556,0.966354,0.888939,0.973236,-0.999220,-0.098744,0.244070,-0.908179,-0.992535,-0.951213,0.857416,0.884046,-0.302529,0.083937,0.309753,0.155213,-0.268512,-0.580742,0.897826,0.345088,-0.590819,0.291470,-0.483769,-0.069371,-0.187219,0.712113,0.125628,0.703442,-0.999992,-0.790992,-0.432383,0.373463,-0.536212,-0.451414,-0.909654,0.402993,0.863570,-0.878168,-0.093176
4,0.678582,-0.376147,-0.843641,0.694254,-0.900952,0.953556,-0.938667,-0.991950,0.496307,0.808705,-0.942533,-0.370675,-0.996992,0.010659,0.914199,0.880765,-0.326852,0.493251,0.566843,0.434197,-0.795550,0.019407,-0.097109,0.947966,-0.965563,-0.053950,-0.605320,0.461053,-0.627470,0.628641,-0.222813,-0.006538,0.500903,-0.189467,0.226259,0.186070,0.854038,0.332476,-0.028554,-0.933556,...,-0.974301,-0.946021,0.968107,0.881028,0.590155,-0.998783,0.010058,-0.113294,-0.799679,-0.974661,-0.955098,0.917061,0.590536,-0.080653,0.847264,-0.542974,-0.586102,0.351221,-0.110621,0.895337,0.288248,-0.628565,0.303319,0.148972,0.414784,-0.219116,0.581784,0.140257,0.5410

In [ ]:
df_weights.iloc[0,]

0      0.507860
1      0.052226
2      0.252734
3      0.219285
4     -0.425712
         ...   
763   -0.778416
764   -0.260559
765    0.919651
766   -0.913012
767    0.303546
Name: 0, Length: 768, dtype: float64

In [ ]:
#saving the bert fine-tuned weights 
df_weights.to_csv('/content/drive/MyDrive/semeval/transformers/bert-weights.csv', index=False)

In [ ]:
# np.random.seed(112)
# df_train, df_valid, df_testing = np.split(df.sample(frac=1, random_state=42), 
#                                      [int(.8*len(df)), int(.9*len(df))])

# print(len(df_train),len(df_val), len(df_test))


8000 1000 100


In [ ]:
len(df)

10000